In [1]:
import numpy as np
import pandas as pd
import get_data
from scipy.spatial.distance import cdist

In [33]:
def get_top_vineyard(desired, lands, k=3):
    desired = desired.drop(["x", "y"], axis=0)
    lands = lands.drop(["x", "y"], axis=1)
    desired = np.array([np.array(desired, dtype=float)])
    lands = np.array(lands, dtype=float)
    distances = cdist(desired, lands)[0]
    closest_lands = lands[np.argpartition(distances, k)[:k]]
    distances = cdist(desired, closest_lands)[0]
    return np.argsort(distances)

In [34]:
vine = pd.read_csv("data.csv")

In [35]:
lands = pd.read_csv("lands.csv")

In [36]:
lands.iloc[0]

y                          403
x                         1721
admin                     True
landcover                   11
relief_aspect            360.0
relief_height                2
relief_slope          0.286477
relief_soiltexture           6
sunny_days                 124
vineyard                 False
water_seasonality            0
Name: 0, dtype: object

In [37]:
np.array(vine.iloc[0], dtype=float)

array([1.9700000e+02, 2.4450000e+03, 0.0000000e+00, 7.0000000e+00,
       3.6000000e+02, 1.7000000e+01, 1.4320962e+00, 3.0000000e+00,
       1.1100000e+02, 1.0000000e+00, 0.0000000e+00])

In [38]:
get_top_vineyard(vine.iloc[0], lands)

array([1, 0, 2])

In [40]:
areas = pd.read_csv("land_area.csv")

In [41]:
areas.head()

,x,y,x_left,x_right,y_top,y_bottom
0,1721,403,1719,1723,400,407
1,1726,403,1724,1729,400,407
2,1732,403,1730,1734,400,407
3,1737,403,1735,1739,400,407
4,1742,403,1740,1744,400,407


In [64]:
def find_closest(wineyards_x, wineyards_y, k):
    wineyards = pd.read_csv("data.csv")
    lands = pd.read_csv("lands.csv")
    desired_wineyards = pd.DataFrame(columns=lands.columns)
    for i in range(len(wineyards_x)):
        s = wineyards.where(wineyards.x == wineyards_x[i]).where(wineyards.y == wineyards_y[i]).iloc[0]
        desired_wineyards = desired_wineyards.append(s)
        
    des = desired_wineyards.mean()
    # print(des)
    needed_indices = get_top_vineyard(des, lands, k)
    areas = pd.read_csv("land_area.csv")
    ans = {
        "x_center": [],
        "y_center": [],
        "x_left": [],
        "x_right": [],
        "y_top": [],
        "y_bottom": [],
    }
    for i in range(k):
        area = areas[(areas.x == lands.iloc[needed_indices[i]].x) & (areas.y == lands.iloc[needed_indices[i]].y)]
        ans["x_center"].append(int(area.x))
        ans["y_center"].append(int(area.y))
        ans["x_left"].append(int(area.x_left))
        ans["x_right"].append(int(area.x_right))
        ans["y_top"].append(int(area.y_top))
        ans["y_bottom"].append(int(area.y_bottom))
    return ans

In [65]:
find_closest([197], [2445], 5)

{'x_center': [1737, 1732, 1726, 1721, 1742],
 'y_center': [403, 403, 403, 403, 403],
 'x_left': [1735, 1730, 1724, 1719, 1740],
 'x_right': [1739, 1734, 1729, 1723, 1744],
 'y_top': [400, 400, 400, 400, 400],
 'y_bottom': [407, 407, 407, 407, 407]}